In [1]:
import os
import torch
import torchvision
from torch import nn
import numpy as np
from tqdm.auto import tqdm
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, datasets
from pathlib import Path
from timeit import default_timer as timer
from torch.utils.data import Subset


In [2]:
pwd

'/kaggle/working'

In [3]:
torch.manual_seed(42)
torch.cuda.manual_seed(42)

device = "cuda" if torch.cuda.is_available() else "cpu"
device



'cuda'

In [4]:
train_dir = Path( '/kaggle/input/super-data/Super_Dataset/train')
test_dir = Path('/kaggle/input/super-data/Super_Dataset/val')


In [5]:
data_transform = transforms.Compose([
    # Resize our images to 64x64
    transforms.Resize(size=(320, 320)),
    # Flip the images randomly on the horizontal 
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(20),
    #transforms.RandomInvert(0.4),
    # Turn the image into a torch.Tensor
    transforms.ToTensor()
])


In [6]:
batch_size = 8
train_dataset = datasets.ImageFolder(train_dir, transform=data_transform, target_transform=None)
test_dataset = datasets.ImageFolder(test_dir, transform=data_transform)


In [7]:
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=os.cpu_count())

test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=os.cpu_count())



In [8]:
class_names= train_dataset.classes
class_names

['Age_Macular_Degeneration',
 'Cataract',
 'Diabetic Retinopathy',
 'Glaucoma',
 'Myopia',
 'Normal']

In [9]:
import torch

from tqdm.auto import tqdm
from typing import Dict, List, Tuple

def train_step(model: torch.nn.Module, 
               dataloader: torch.utils.data.DataLoader, 
               loss_fn: torch.nn.Module, 
               optimizer: torch.optim.Optimizer,
               device: torch.device) -> Tuple[float, float]:

  # Put model in train mode
  model.train()

  # Setup train loss and train accuracy values
  train_loss, train_acc = 0, 0

  # Loop through data loader data batches
  for batch, (X, y) in enumerate(dataloader):
      # Send data to target device
      X, y = X.to(device), y.to(device)

      # 1. Forward pass
      y_pred = model(X)

      # 2. Calculate  and accumulate loss
      loss = loss_fn(y_pred, y)
      train_loss += loss.item() 

      # 3. Optimizer zero grad
      optimizer.zero_grad()

      # 4. Loss backward
      loss.backward()

      # 5. Optimizer step
      optimizer.step()

      # Calculate and accumulate accuracy metric across all batches
      y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
      train_acc += (y_pred_class == y).sum().item()/len(y_pred)

  # Adjust metrics to get average loss and accuracy per batch 
  train_loss = train_loss / len(dataloader)
  train_acc = train_acc / len(dataloader)
  return train_loss, train_acc

def test_step(model: torch.nn.Module, 
              dataloader: torch.utils.data.DataLoader, 
              loss_fn: torch.nn.Module,
              device: torch.device) -> Tuple[float, float]:

  # Put model in eval mode
  model.eval() 

  # Setup test loss and test accuracy values
  test_loss, test_acc = 0, 0

  # Turn on inference context manager
  with torch.inference_mode():
      # Loop through DataLoader batches
      for batch, (X, y) in enumerate(dataloader):
          # Send data to target device
          X, y = X.to(device), y.to(device)

          # 1. Forward pass
          test_pred_logits = model(X)

          # 2. Calculate and accumulate loss
          loss = loss_fn(test_pred_logits, y)
          test_loss += loss.item()

          # Calculate and accumulate accuracy
          test_pred_labels = test_pred_logits.argmax(dim=1)
          test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))

  # Adjust metrics to get average loss and accuracy per batch 
  test_loss = test_loss / len(dataloader)
  test_acc = test_acc / len(dataloader)
  return test_loss, test_acc

def train(model: torch.nn.Module, 
          train_dataloader: torch.utils.data.DataLoader, 
          test_dataloader: torch.utils.data.DataLoader, 
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device) -> Dict[str, List]:
          
  # Create empty results dictionary
  results = {"train_loss": [],
      "train_acc": [],
      "test_loss": [],
      "test_acc": []
  }

  # Loop through training and testing steps for a number of epochs
  for epoch in tqdm(range(epochs)):
      train_loss, train_acc = train_step(model=model,
                                          dataloader=train_dataloader,
                                          loss_fn=loss_fn,
                                          optimizer=optimizer,
                                          device=device)
      test_loss, test_acc = test_step(model=model,
          dataloader=test_dataloader,
          loss_fn=loss_fn,
          device=device)

      # Print out what's happening
      print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"
      )

      # Update results dictionary
      results["train_loss"].append(train_loss)
      results["train_acc"].append(train_acc)
      results["test_loss"].append(test_loss)
      results["test_acc"].append(test_acc)

  # Return the filled results at the end of the epochs
  return results

In [10]:
from typing import List, Tuple

from PIL import Image
def pred_and_plot_image(model: torch.nn.Module,
                        image_path: str, 
                        class_names: List[str],
                        image_size: Tuple[int, int] = (512, 512),
                        transform: torchvision.transforms = None,
                        device: torch.device=device):
    
    
    # 2. Open image
    img = Image.open(image_path)

    # 3. Create transformation for image (if one doesn't exist)
    if transform is not None:
        image_transform = data_transform
    else:
        image_transform = transforms.Compose([
            transforms.Resize(image_size),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225]),
        ])

    ### Predict on image ### 

    # 4. Make sure the model is on the target device
    model.to(device)

    # 5. Turn on model evaluation mode and inference mode
    model.eval()
    with torch.inference_mode():
      # 6. Transform and add an extra dimension to image (model requires samples in [batch_size, color_channels, height, width])
      transformed_image = image_transform(img).unsqueeze(dim=0)

      # 7. Make a prediction on image with an extra dimension and send it to the target device
      target_image_pred = model(transformed_image.to(device))

    # 8. Convert logits -> prediction probabilities (using torch.softmax() for multi-class classification)
    target_image_pred_probs = torch.softmax(target_image_pred, dim=1)

    # 9. Convert prediction probabilities -> prediction labels
    target_image_pred_label = torch.argmax(target_image_pred_probs, dim=1)

    # 10. Plot image with predicted label and probability 
    
    print(f"Pred: {class_names[target_image_pred_label]} | Prob: {target_image_pred_probs.max():.3f}")


In [11]:
weights = torchvision.models.EfficientNet_B6_Weights.IMAGENET1K_V1

model = torchvision.models.efficientnet_b6(weights=weights).to(device)

for params in model.parameters():
    params.requires_grad = True

# n_inputs = model.fc.in_features
model.classifier = nn.Sequential(nn.Dropout(p=0.4),
                                nn.Linear(2304, 2304),
                                nn.ReLU(),
                                nn.Dropout(p=0.4),
                                nn.Linear(2304, 6),
                                nn.LogSigmoid()).to(device)

                               
for name, child in model.named_children():
    for name2, params in child.named_parameters():
        params.requires_grad = True

weights = [1/186, 1/793, 1/1893, 1/773, 1/162, 1/2095]
weights = torch.FloatTensor(weights).cuda()
loss_fn = nn.CrossEntropyLoss(weight=weights)
optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-4, weight_decay=1e-4)

epochs = 16
train_start = timer()
model_results = train(model,train_dataloader, test_dataloader, optimizer, loss_fn, epochs, device)
train_end = timer()

print(f"Total time on {device}: {train_end - train_start}")

# custom_image_path = Path('/content/drive/MyDrive/NF.jpg')

# pred_and_plot_image(model=model, image_path=custom_image_path, class_names= class_names)

save_path = Path("/kaggle/working/Super_model_29_04_2023.EfficientnetB6_V2_8-Batch.pt")

torch.save(model,save_path)


Downloading: "https://download.pytorch.org/models/efficientnet_b6_lukemelas-c76e70fd.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b6_lukemelas-c76e70fd.pth


  0%|          | 0.00/165M [00:00<?, ?B/s]

  0%|          | 0/16 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 0.9729 | train_acc: 0.6060 | test_loss: 0.8308 | test_acc: 0.6703
Epoch: 2 | train_loss: 0.6071 | train_acc: 0.7357 | test_loss: 0.7250 | test_acc: 0.7200
Epoch: 3 | train_loss: 0.4810 | train_acc: 0.7829 | test_loss: 0.6680 | test_acc: 0.7572
Epoch: 4 | train_loss: 0.4205 | train_acc: 0.7976 | test_loss: 0.6407 | test_acc: 0.7635
Epoch: 5 | train_loss: 0.3797 | train_acc: 0.8115 | test_loss: 0.7181 | test_acc: 0.7355
Epoch: 6 | train_loss: 0.3448 | train_acc: 0.8289 | test_loss: 0.5587 | test_acc: 0.7965
Epoch: 7 | train_loss: 0.2868 | train_acc: 0.8498 | test_loss: 0.7109 | test_acc: 0.7942
Epoch: 8 | train_loss: 0.2632 | train_acc: 0.8597 | test_loss: 0.6034 | test_acc: 0.7971
Epoch: 9 | train_loss: 0.2769 | train_acc: 0.8631 | test_loss: 0.6172 | test_acc: 0.8008
Epoch: 10 | train_loss: 0.2382 | train_acc: 0.8806 | test_loss: 0.7192 | test_acc: 0.7904
Epoch: 11 | train_loss: 0.2103 | train_acc: 0.8958 | test_loss: 0.7788 | test_acc: 0.7744
Epoch: 12 | train_l

In [21]:
# model = torch.load("/content/drive/MyDrive/Multiple_model_25_04_2023.Resnet_V2_8-Batch.pt")
# custom_image_path = Path('/content/drive/MyDrive/MYO.jpg')

# pred_and_plot_image(model=model, image_path=custom_image_path, class_names= class_names )

Pred: Normal | Prob: 1.000


In [12]:
import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torchvision.models import resnet18
from sklearn.metrics import confusion_matrix

val_dir= '/kaggle/input/data-1/Super_Dataset/test'
test_dataset = datasets.ImageFolder(test_dir, transform=data_transform)

# Load the pretrained ResNet18 model
# model = torch.load("/content/drive/MyDrive/Multiple_model_25_04_2023.Resnet_V2_8-Batch.pt")

# Set device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the device
model.to(device)

# Evaluate the model on the test dataset
model.eval()
predictions = []
true_labels = []
with torch.no_grad():
    for inputs, labels in test_dataset:
        inputs = inputs.to(device)
        labels = torch.tensor(labels).to(device)
        outputs = model(inputs.unsqueeze(0))
        _, predicted = torch.max(outputs.data, 1)
        predictions.extend(predicted.cpu().numpy())
        true_labels.extend((labels.item(),))

# Compute the confusion matrix
cm = confusion_matrix(true_labels, predictions)

# Compute the accuracy
accuracy = cm.diagonal().sum() / cm.sum()

print("Confusion matrix:")
print(cm)
print("Accuracy:", accuracy)

Confusion matrix:
[[ 39   0   2   0   0  12]
 [  0 238   0  11   0   3]
 [ 14   2 406  16   3  99]
 [  2   7   0 231   0   6]
 [  0   0   0   0  42   4]
 [ 27  26  66  98   2 495]]
Accuracy: 0.7839005942733658
